In [ ]:
plt.close('all')
durations = fix_df.duration.values
plt.hist(durations[y_pred==0],bins=np.arange(0,1000,10),label='mode 0') # choice fixation
plt.hist(durations[y_pred==1],bins=np.arange(0,1000,10),label='mode 1') # short fixation
plt.hist(durations[y_pred==2],bins=np.arange(0,1000,10),label='mode 2') # planning fixation
plt.hist(durations[y_pred==3],bins=np.arange(0,1000,10),label='mode 3') # long fixation
plt.ylabel('counts')
plt.title('Fixation duration (ms)')
plt.legend()

In [ ]:
def plot_psth2(ax,var,df,spikes): 

    for node in np.arange(0, 16):
        
        idx = df[var].values == node
        x, y = xy(node, 4, 4)
        trace = movmean(np.mean(spikes[idx,:,int(nid)],axis=0)*1000, w=75).flatten()
        t = np.arange(-500, 500)
        ax.plot(t,trace,c= cmap_2d[x, y])
        ax.axvline(0,color='k',linestyle='--',lw=0.8)
        ax.set_title(var)

In [ ]:
def get_action(prenode,node):
    
    x1, y1 = xy(prenode, 4, 4)
    x2, y2 = xy(node, 4, 4)

    if prenode==-1:
        
        action = -1
        
    else:

        if x1 == x2:
            if y2-y1 == 1:
                action = 0
            if y2-y1 == -1:
                action = 1
                
        if y1 == y2:
            if x2-x1 == 1:
                action = 2
            if x2-x1 == -1:
                action = 3
    
        if (prenode==0) & (node==15):
            action = 4
    
        if (prenode==15) & (node==0):
            action = 5

    return action

In [ ]:
def create_balanced_datasets(df,FR,var1,var2,trialNum=100):
    
    bstrap_df = pd.DataFrame()
    bstrap_spikes = []
    
    for var1_ind,val1 in enumerate(np.unique(df[var1])):  
        for var2_ind,val2 in enumerate(np.unique(df[var2])):
    
            condition_idx = (df[var1]==val1) & (df[var2]==val2)
            temp_df = df[condition_idx].reset_index(drop=True)  
            temp_FR = FR[condition_idx,:,:]
    
            if len(temp_df)>10:
                
                idx = np.random.randint(len(temp_df),size=trialNum)
                bstrap_df = pd.concat([bstrap_df,temp_df.loc[idx,:]])
                bstrap_spikes.append(temp_FR[idx,:,:])

    return bstrap_df, np.concatenate(bstrap_spikes)

In [ ]:
def plot_task_grid(fig, ax, marker_params = [50, 4], step_length=4.35): 
""" Bart Vprobe: step_length = 4.35
Bart + London NPX: step length = 5.4
"""
node_x = -((5 / 2) - np.arange(1, 5)) * step_length
node_y = -(np.arange(1, 5) - 5 / 2) * step_length
node_centers = np.zeros([2, 16])
for i, (y_center, x_center) in enumerate(product(node_y, node_x)):
node_centers[:, i] = [x_center, y_center]
ax.plot(node_centers[0, :], node_centers[1, :], marker="o", linestyle="none", c=[0.5, 0.5, 0.5], fillstyle="none", markersize=marker_params[0],
markeredgewidth=marker_params[1])
ax.set_xlim([-12, 12])
ax.set_ylim([-12, 12])
return fig, ax

In [ ]:
nwbfile = pynwb.NWBHDF5IO("Bart_TeleWorld_v13_033123.nwb", "r").read()
fix_df = nwbfile.intervals["fixations"].to_dataframe()
fix_df = fix_df[fix_df.duration>50]
fix_df['planning'] = (fix_df.duration<300) & (fix_df.active_prob<0.2)
fix_df['choice'] = (fix_df.duration>300) & (fix_df.active_prob>0.2)
choice_fix = fix_df.query("choice")
interval = choice_fix.start_time[1:].values-choice_fix.stop_time[:-1].values
plt.close('all')
plt.hist(interval[interval<200],bins=np.arange(0,200,10))